In [22]:
import pandas as pd
import numpy as np

from unicode import join_jamos
from jamo import j2hcj
import textgrid

from tqdm import tqdm
from glob import glob

import librosa
import soundfile as sf

In [23]:
wavs = glob('data/kss/*.wav')
textgrids = glob('TextGrid/*.TextGrid')

for i in tqdm(range(len(wavs))):
    wav = librosa.load(wavs[i], sr = 22050)[0]
    grid = textgrid.TextGrid.fromFile(f"TextGrid/{wavs[i].split('/')[-1].split('.')[0]}.TextGrid")
    pha = grid[1]
    if pha[0].mark == '':
        dur = pha[0].duration()
        wav = wav[round(dur * 22050):]
    else:
        pass
    sf.write(wavs[i], wav, 22050)

100%|██████████| 3868/3868 [00:18<00:00, 204.48it/s]


In [ ]:
df = pd.read_csv('data/transcript.v.1.4.txt', sep = '|', header = None)
df.columns = ['name', 'sentence1', 'sentence2', 'jamo', 'length']
df = df.set_index('name')
textgrid_path = glob('TextGrid/*')

for i in tqdm(range(len(textgrid_path))):
    text_grid = textgrid.TextGrid.fromFile(textgrid_path[i])
    
    pha = text_grid[1]
    phs = []
    for iterator, interval in enumerate(pha.intervals):
        if interval.mark == "":
            interval.mark = " "
        mark = interval.mark
        phs.append(mark)

    x = join_jamos(j2hcj("".join(phs)))
    x = x.strip()
    a = df.loc[textgrid_path[i].split('/')[-1].replace('TextGrid', 'wav')]['sentence2']
    df['sentence2'] = df['sentence2'].replace(a, x)
df

In [ ]:
df.reset_index(inplace = True)
df.to_csv('data/transcript.v.1.4.txt', sep = '|', header = None, index = False)

In [ ]:
data = glob('dump/train/fix_dur/*')

for i in range(len(data)):
    a = len(np.load(data[i]))
    b = len(np.load(f"dump/train/ids/{data[i].split('/')[-1].split('-')[0]}-ids.npy"))
    if a == b:
        pass
    else:
        print(data[i])